In [ ]:
# | default_exp _code_generator.helper

In [ ]:
# | export

from typing import *
import random
import time
from contextlib import contextmanager

import openai
from fastcore.foundation import patch

from fastkafka._components.logger import get_logger
from fastkafka._code_generator.prompts import SYSTEM_PROMPT, DEFAULT_FASTKAFKA_PROMPT

In [ ]:
import pytest
import unittest.mock

from fastkafka._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export

DEFAULT_PARAMS = {
    "temperature": 0.7,
}

DEFAULT_MODEL = "gpt-3.5-turbo-16k" # gpt-3.5-turbo

MAX_RETRIES = 5

In [ ]:
# | export

# Reference: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_handle_rate_limits.ipynb


def _retry_with_exponential_backoff(
    initial_delay: float = 1,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 10,
    max_wait: float = 60,
    errors: tuple = (
        openai.error.RateLimitError,
        openai.error.ServiceUnavailableError,
        openai.error.APIError,
    ),
) -> Callable:
    """Retry a function with exponential backoff."""

    def decorator(
        func: Callable[[str], Tuple[str, str]]
    ) -> Callable[[str], Tuple[str, str]]:
        def wrapper(*args, **kwargs):  # type: ignore
            num_retries = 0
            delay = initial_delay

            while True:
                try:
                    return func(*args, **kwargs)

                except errors as e:
                    num_retries += 1
                    if num_retries > max_retries:
                        raise Exception(
                            f"Maximum number of retries ({max_retries}) exceeded."
                        )
                    delay = min(
                        delay
                        * exponential_base
                        * (1 + jitter * random.random()),  # nosec
                        max_wait,
                    )
                    logger.info(
                        f"Note: OpenAI's API rate limit reached. Command will automatically retry in {int(delay)} seconds. For more information visit: https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits",
                    )
                    time.sleep(delay)

                except Exception as e:
                    raise e

        return wrapper

    return decorator

In [ ]:
@_retry_with_exponential_backoff()
def mock_func():
    return "Success"

actual = mock_func()
expected = "Success"

print(actual)
assert actual == expected

Success


In [ ]:
# Test max retries exceeded
@_retry_with_exponential_backoff(max_retries=1)
def mock_func_error():
    raise openai.error.RateLimitError


with pytest.raises(Exception) as e:
    mock_func_error()

print(e.value)
assert str(e.value) == "Maximum number of retries (1) exceeded."

[INFO] __main__: Note: OpenAI's API rate limit reached. Command will automatically retry in 2 seconds. For more information visit: https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits
Maximum number of retries (1) exceeded.


In [ ]:
# | export


class CustomAIChat:
    """Custom class for interacting with OpenAI

    Attributes:
        model: The OpenAI model to use. If not passed, defaults to gpt-3.5-turbo-16k.
        system_prompt: Initial system prompt to the AI model. If not passed, defaults to SYSTEM_PROMPT.
        initial_user_prompt: Initial user prompt to the AI model.
        params: Parameters to use while initiating the OpenAI chat model. DEFAULT_PARAMS used if not provided.
    """

    def __init__(
        self,
        model: Optional[str] = DEFAULT_MODEL,
        user_prompt: Optional[str] = None,
        params: Dict[str, float] = DEFAULT_PARAMS,
    ):
        """Instantiates a new CustomAIChat object.

        Args:
            model: The OpenAI model to use. If not passed, defaults to gpt-3.5-turbo-16k.
            user_prompt: The user prompt to the AI model.
            params: Parameters to use while initiating the OpenAI chat model. DEFAULT_PARAMS used if not provided.
        """
        self.model = model
        self.messages = [
            {"role": role, "content": content}
            for role, content in [
                ("system", SYSTEM_PROMPT),
                ("user", DEFAULT_FASTKAFKA_PROMPT),
                ("user", user_prompt),
            ]
            if content is not None
        ]
        self.params = params

    @_retry_with_exponential_backoff()
    def __call__(self, user_prompt: str) -> Tuple[str, str]:
        """Call OpenAI API chat completion endpoint and generate a response.

        Args:
            user_prompt: A string containing user's input prompt.

        Returns:
            A tuple with AI's response message content and the total number of tokens used while generating the response.
        """
        self.messages.append(
            {"role": "user", "content": f"==== APP DESCRIPTION: ====\n\n{user_prompt}"}
        )
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=self.messages,
            temperature=self.params["temperature"],
        )
        return (
            response["choices"][0]["message"]["content"],
            response["usage"]["total_tokens"],
        )

In [ ]:
TEST_INITIAL_USER_PROMPT = """
You should respond with 0, 1 or 2 and nothing else. Below are your rules:

==== RULES: ====

If the ==== APP DESCRIPTION: ==== section is not related to FastKafka or contains violence, self-harm, harassment/threatening or hate/threatening information then you should respond with 0.

If the ==== APP DESCRIPTION: ==== section is related to FastKafka but focuses on what is it and its general information then you should respond with 1. 

If the ==== APP DESCRIPTION: ==== section is related to FastKafka but focuses how to use it and instructions to create a new app then you should respond with 2. 
"""

ai = CustomAIChat(user_prompt = TEST_INITIAL_USER_PROMPT)
response, total_tokens = ai("Name the tallest mountain in the world")

print(response)
print(total_tokens)

assert response == "0"

0
2136


In [ ]:

@contextmanager
def mock_openai_create(test_response):
    mock_choices = {
        "choices": [{"message": {"content": test_response}}],
        "usage": {"total_tokens": 100},
    }

    with unittest.mock.patch("openai.ChatCompletion") as mock:
        mock.create.return_value = mock_choices
        yield

In [ ]:
test_response = "This is a mock response"

with mock_openai_create(test_response):
    response = openai.ChatCompletion.create()
    ret_val = response['choices'][0]['message']['content']
    print(ret_val)
    assert ret_val == test_response

This is a mock response


In [ ]:
# | export


class ValidateAndFixResponse:
    """Generates and validates response from OpenAI

    Attributes:
        generate: A callable object for generating responses.
        validate: A callable object for validating responses.
        max_attempts: An optional integer specifying the maximum number of attempts to generate and validate a response.
    """

    def __init__(
        self,
        generate: Callable[..., Any],
        validate: Callable[..., Any],
        max_attempts: Optional[int] = MAX_RETRIES,
    ):
        self.generate = generate
        self.validate = validate
        self.max_attempts = max_attempts

    def construct_prompt_with_error_msg(
        self,
        prompt: str,
        response: str,
        errors: str,
    ) -> str:
        """Construct prompt message along with the error message.

        Args:
            prompt: The original prompt string.
            response: The invalid response string from OpenAI.
            errors: The errors which needs to be fixed in the invalid response.

        Returns:
            A string combining the original prompt, invalid response, and the error message.
        """
        prompt_with_errors = (
            prompt
            + f"\n\n==== RESPONSE WITH ISSUES ====\n\n{response}"
            + f"\n\nRead the contents of ==== RESPONSE WITH ISSUES ==== section and fix the below mentioned issues:\n\n{errors}"
        )
        return prompt_with_errors

    def fix(self, prompt: str) -> Tuple[str, str]:
        raise NotImplementedError()

In [ ]:
def fixture_generate(initial_prompt):
    return "some response"

def fixture_validate(response):
    return []

prompt = "some prompt"
response = "some response"
errors = """error 1
error 2
error 3
"""

expected = """some prompt

==== RESPONSE WITH ISSUES ====

some response

Read the contents of ==== RESPONSE WITH ISSUES ==== section and fix the below mentioned issues:

error 1
error 2
error 3
"""

fix_response = ValidateAndFixResponse(fixture_generate, fixture_validate)
actual = fix_response.construct_prompt_with_error_msg(prompt, response, errors)
print(actual)

assert actual == expected

some prompt

==== RESPONSE WITH ISSUES ====

some response

Read the contents of ==== RESPONSE WITH ISSUES ==== section and fix the below mentioned issues:

error 1
error 2
error 3



In [ ]:
# | export


@patch # type: ignore
def fix(self: ValidateAndFixResponse, prompt: str) -> Tuple[str, str]:
    """Fix the response from OpenAI until no errors remain or maximum number of attempts is reached.

    Args:
        prompt: The initial prompt string.

    Returns:
        str: The generated response that has passed the validation.

    Raises:
        ValueError: If the maximum number of attempts is exceeded and the response has not successfully passed the validation.
    """
    iterations = 0
    initial_prompt = prompt
    while True:
        response, total_tokens = self.generate(prompt)
        errors = self.validate(response)
        if len(errors) == 0:
            return response, total_tokens
        error_str = "\n".join(errors)
        prompt = self.construct_prompt_with_error_msg(
            initial_prompt, response, error_str
        )
        iterations += 1
        if self.max_attempts is not None and iterations >= self.max_attempts:
            raise ValueError(error_str)

In [ ]:
fixture_initial_prompt = "some valid prompt"
expected = "Some Valid response"

def fixture_generate(initial_prompt):
    return expected, 2

def fixture_validate(response):
    return []

v = ValidateAndFixResponse(fixture_generate, fixture_validate)
actual, tokens = v.fix(fixture_initial_prompt)
print(actual)
assert actual == expected

Some Valid response


In [ ]:
fixture_initial_prompt = "some invalid prompt"
max_attempts = 2

def fixture_generate(initial_prompt):
    return "some invalid response", 2

def fixture_validate(response):
    return ["error 1", "error 2"]

expected = """error 1
error 2"""

with pytest.raises(ValueError) as e:
    v = ValidateAndFixResponse(fixture_generate, fixture_validate, max_attempts)
    actual = v.fix(fixture_initial_prompt)
print(e.value)
assert str(e.value) == expected

error 1
error 2
